# Definition and introduction

Descriptors are Python objects that implement a method of the descriptor protocol, which gives you the ability to create objects that have special behavior when they’re accessed as attributes of other objects. 

### Descriptor protocol

Object defining any of the below method considered as descriptor and can override default behavior upon being looked up as an attribute.

1. ``descr.__get__(self, obj, type=None)``

2. ``descr.__set__(self, obj, value)``

3. ``descr.__delete__(self, obj)``

<br><br>
Descriptor can optionally implement the ``__set_name__(self, owner, name)`` method.

### Data vs. Non-data Descriptors

1. Data descriptors are objects of a class that implements ``__set__ ``method (and/or ``__delete__`` method)
2. Non-data descriptors are objects of a class that have the ``__get__`` method only.


The ``__set_name__`` method doesn’t affect the classification of the descriptors. 

The descriptor types determine how Python resolves object’s attributes lookup.

Following example defines a descriptor that logs something on the console when it’s accessed:

In [3]:
from ipywidgets import HTML

display(HTML("<h3>Output:</h3>"))
# descriptors.py
class Verbose_attribute():
    def __get__(self, obj, type=None) -> object:
        print("accessing the attribute to get the value")
        return 42
    def __set__(self, obj, value) -> None:
        print("accessing the attribute to set the value")
        raise AttributeError("Cannot change the value")

class Foo():
    attribute1 = Verbose_attribute()

my_foo_object = Foo()
x = my_foo_object.attribute1


print(x)

HTML(value='<h3>Output:</h3>')

accessing the attribute to get the value
42


In the example above, ``Verbose_attribute()`` implements the descriptor protocol. Once it’s instantiated as an attribute of Foo, it can be considered a descriptor.

As a descriptor, it has **binding behavior** when it’s accessed using dot notation. In this case, the descriptor logs a message on the console every time it’s accessed to get or set a value:

1. When it’s accessed to .``__get__()`` the value, it always returns the value 42.
2. When it’s accessed to .``__set__()`` a specific value, it raises an AttributeError exception, which is the recommended way to implement read-only descriptors.

### How Attributes Are Accessed With the Lookup Chain

 When we access the attribute of an object with dot notation, here’s how Python interpreter knows using a process called the **lookup chain**. 

 Below is the **lookup chain** process:-

1. First, we’ll get the result returned from the ``__get__`` method of the data descriptor named after the attribute we’re looking for.

2. If that fails, then we’ll get the value of the object’s ``__dict__`` for the key named after the attribute we’re looking for.

3. If that fails, then we’ll get the result returned from the ``__get__`` method of the non-data descriptor named after the attribute we’re looking for.

4. If that fails, then we’ll get the value of the object type’s ``__dict__`` for the key named after the attribute we’re looking for.

5. If that fails, then we’ll get the value of the object parent type’s ``__dict__`` for the key named after the attribute we’re looking for.

6. If that fails, then the previous step is repeated for all the parent’s types in the method resolution order of the object.

7. If everything else has failed, then we’ll get an **AttributeError** exception.


### How to Use Python Descriptors Properly

The most important methods of this protocol are .``__get__()`` and .``__set__()``, which have the following signature:

```python
__get__(self, obj, type=None)
__set__(self, obj, value)
```

**Things to keep in mind:**

1. **self** is the instance of the descriptor you’re writing.
2. **obj** is the instance of the object your descriptor is attached to.
3. **type** is the type of the object the descriptor is attached to.

>**Note:**In ``.__set__()``, we don’t have the ``type`` variable, because we can only call ``.__set__()`` on the object. In contrast, we can call ``.__get__()`` on both the object and the class.

Another important thing to know is that Python descriptors are instantiated just once per class. That means that every single instance of a class containing a descriptor shares that descriptor instance. This is something that we might not expect and can lead to a classic pitfall, like this:

In [4]:
from ipywidgets import HTML

display(HTML("<h3>Output:</h3>"))

# descriptors2.py
class OneDigitNumericValue():
    def __init__(self):
        self.value = 0
    def __get__(self, obj, type=None) -> object:
        return self.value
    def __set__(self, obj, value) -> None:
        if value > 9 or value < 0 or int(value) != value:
            raise AttributeError("The value is invalid")
        self.value = value

class Foo():
    number = OneDigitNumericValue()

my_foo_object = Foo()
my_second_foo_object = Foo()

my_foo_object.number = 3
print(my_foo_object.number)
print(my_second_foo_object.number)

my_third_foo_object = Foo()
print(my_third_foo_object.number)

HTML(value='<h3>Output:</h3>')

3
3
3


We can see that all the instances of ``Foo`` have the same value for the attribute ``number``.

The best solution here is to simply not store values in the descriptor itself, but to store them in the object that the descriptor is attached to as shown below:

In [8]:
# descriptors4.py
class OneDigitNumericValue():
    def __init__(self, name):
        self.name = name

    def __get__(self, obj, type=None) -> object:
        return obj.__dict__.get(self.name)or 0

    def __set__(self, obj, value) -> None:
        obj.__dict__[self.name] = value

class Foo():
    number = OneDigitNumericValue("number")

my_foo_object = Foo()
my_second_foo_object = Foo()

display(HTML("<h3>Output:</h3>"))
my_foo_object.number = 3
print(my_foo_object.number)
print(my_second_foo_object.number)

my_third_foo_object = Foo()
print(my_third_foo_object.number)

HTML(value='<h3>Output:</h3>')

3
0
0


In this example, when we set a value to the ``number`` attribute of the object, the descriptor stores it in the ``__dict__`` attribute of the object it’s attached to using the same name of the descriptor itself.

The only problem here is that when we instantiate the descriptor you have to specify the name as a parameter like below :

```python
number = OneDigitNumericValue("number")
```

In Python 3.6 or higher, the descriptor protocol has a new method ``.__set_name__()``. With this new method, whenever we instantiate a descriptor this method is called and the name parameter automatically set.  Rewriting above code using ``._set_name_()``.

In [10]:
# descriptors5.py
class OneDigitNumericValue():
    def __set_name__(self, owner, name):
        self.name = name

    def __get__(self, obj, type=None) -> object:
        return obj.__dict__.get(self.name) or 0

    def __set__(self, obj, value) -> None:
        obj.__dict__[self.name] = value

class Foo():
    number = OneDigitNumericValue()


display(HTML("<h3>Output:</h3>"))

my_foo_object = Foo()
my_second_foo_object = Foo()

my_foo_object.number = 3
print(my_foo_object.number)
print(my_second_foo_object.number)

my_third_foo_object = Foo()
print(my_third_foo_object.number)

HTML(value='<h3>Output:</h3>')

3
0
0


Now, ``.__init__()`` has been removed and ``.__set_name__()`` has been implemented. This makes it possible to create descriptor without specifying the name of the internal attribute that we need to use for storing the value.

## Why Use Python Descriptors?

#### D.R.Y. Code

Suppose we have a class ``Person`` with two instance attributes ``first_name`` and ``last_name``and we want the ``first_name`` and ``last_name`` attributes to be non-empty strings. To enforce the data validity, we can use property with a ``getter`` and ``setter`` methods, like this:

```python
class Person:
    def __init__(self, first_name, last_name):
        self.first_name = first_name
        self.last_name = last_name

    @property
    def first_name(self):
        return self._first_name

    @first_name.setter
    def first_name(self, value):
        if not isinstance(value, str):
            raise ValueError('The first name must a string')

        if len(value) == 0:
            raise ValueError('The first name cannot be empty')

        self._first_name = value

    @property
    def last_name(self):
        return self._last_name

    @last_name.setter
    def last_name(self, value):
        if not isinstance(value, str):
            raise ValueError('The last name must a string')

        if len(value) == 0:
            raise ValueError('The last name cannot be empty')

        self._last_name = value

```

This code works perfectly fine. However, it is redundant because the validation logic validates the ``first`` and ``last names`` is the same.

Also, if the class has more attributes that require a non-empty string, we need to duplicate this logic in other properties. In other words, this validation logic is not reusable.

To avoid duplicating the logic, we may create ``RequiredString`` descriptor will enable reusability across all the propertis of ``Person`` class.

Below is the implementation of ``RequiredString`` descriptor class:

In [11]:
class RequiredString:
    def __set_name__(self, owner, name):
        self.property_name = name

    def __get__(self, instance, owner):
        if instance is None:
            return self

        return instance.__dict__[self.property_name] or None

    def __set__(self, instance, value):
        if not isinstance(value, str):
            raise ValueError(f'The {self.property_name} must be a string')

        if len(value) == 0:
            raise ValueError(f'The {self.property_name} cannot be empty')

        instance.__dict__[self.property_name] = value

Second, use the ``RequiredString`` class in the ``Person`` class:

```python
class Person:
    first_name = RequiredString()
    last_name = RequiredString()
```

f you assign an empty string or a non-string value to the first_name or last_name attribute of the Person class, you’ll get an error.

For example, the following attempts to assign an empty string to the first_name attribute:

```python
try:
    person = Person()
    person.first_name = ''
except ValueError as e:
    print(e)
```

Error:

```
The first_name must be a string

```

Besides the ``RequiredString``, we can define other descriptors to enforce other data types like age, email, and phone. 

Here is the Complete Code:

In [12]:
from ipywidgets import HTML

class RequiredString:
    def __set_name__(self, owner, name):
        print(f'__set_name__ was called with owner={owner} and name={name}')
        self.property_name = name

    def __get__(self, instance, owner):
        print(f'__get__ was called with instance={instance} and owner={owner}')
        if instance is None:
            return self

        return instance.__dict__[self.property_name] or None

    def __set__(self, instance, value):
        print(f'__set__ was called with instance={instance} and value={value}')

        if not isinstance(value, str):
            raise ValueError(f'The {self.property_name} must a string')

        if len(value) == 0:
            raise ValueError(f'The {self.property_name} cannot be empty')

        instance.__dict__[self.property_name] = value

display(HTML("<h3>Ouput:</h3>"))

class Person:
    first_name = RequiredString()
    last_name = RequiredString()
    
try:
    person = Person()
    person.first_name = ''
except ValueError as e:
    print(e)    

HTML(value='<h3>Ouput:</h3>')

__set_name__ was called with owner=<class '__main__.Person'> and name=first_name
__set_name__ was called with owner=<class '__main__.Person'> and name=last_name
__set__ was called with instance=<__main__.Person object at 0x11638a6c0> and value=
The first_name cannot be empty


## What is happening behind the scene?

When Python executes below statements:

```python
first_name = RequiredString()
last_name = RequiredString()
```

it will  creates the descriptor objects for ``first_name`` and ``last_name`` and automatically call the ``__set_name__`` method of these objects. Here’s the output:

```
__set_name__ was called with owner=<class '__main__.Person'> and name=first_name
__set_name__ was called with owner=<class '__main__.Person'> and name=last_name
```

In this example, the ``owner`` argument of ``__set_name__`` is set to the Person class in the __main__ module, and the name argument is set to the first_name and last_name attribute accordingly.

The following statements are equivalent:

```
first_name = RequiredString()
```

and

```
first_name.__set_name__(Person, 'first_name')
```

Inside, the ``__set_name__`` method, we assign the name argument to the ``property_name`` instance attribute of the descriptor object so that we can access it later in the `` __get__`` and ``__set__`` method:

```
self.property_name = name
```

The ``first_name`` and ``last_name`` are the class variables of the ``Person`` class. If we look at the ``Person.__dict__`` class attribute, we’ll see two descriptor objects ``first_name`` and ``last_name``:

In [13]:
from pprint import pprint

pprint(Person.__dict__)

mappingproxy({'__dict__': <attribute '__dict__' of 'Person' objects>,
              '__doc__': None,
              '__module__': '__main__',
              '__weakref__': <attribute '__weakref__' of 'Person' objects>,
              'first_name': <__main__.RequiredString object at 0x11637f4d0>,
              'last_name': <__main__.RequiredString object at 0x11638a780>})


When we assign the new value to a descriptor, Python calls ``__set__`` method to set the attribute on an instance of the owner class to the new value.

In [14]:
person = Person()
person.first_name = 'John'

__set__ was called with instance=<__main__.Person object at 0x11638af90> and value=John


In this example, the instance argument is ``person`` object and the value is the string ``'John'``. Inside the ``__set__`` method, we raise a ``ValueError`` if the new value is not a string or if it is an empty string.

Otherwise, we assign the value to the instance attribute ``first_name`` of the ``person`` object:

```python
instance.__dict__[self.property_name] = value
```

Note that Python uses instance.__dict__ dictionary to store instance attributes of the instance object.

Once we set the ``first_name`` and ``last_name`` of an instance of the ``Person`` object, we’ll see the instance attributes with the same names in the instance’s ``__dict__.`` For example:

In [15]:
person = Person()
print(person.__dict__)  # {}

person.first_name = 'John'
person.last_name = 'Doe'

print(person.__dict__) # {'first_name': 'John', 'last_name': 'Doe'}

{}
__set__ was called with instance=<__main__.Person object at 0x11638ad80> and value=John
__set__ was called with instance=<__main__.Person object at 0x11638ad80> and value=Doe
{'first_name': 'John', 'last_name': 'Doe'}


Python calls the ``__get__`` method of the ``Person‘s`` object when you access the first_name attribute. For example:

```python
person = Person()

person.first_name = 'John'
print(person.first_name)
```

The ``__get__`` method returns the descriptor if the instance is None. For example, if we access the ``first_name`` or ``last_name`` from the Person class, you’ll see the descriptor object:

```python
print(Person.first_name)
```

If the ``instance`` is not ``None``, the ``__get__()`` method returns the value of the attribute with the name ``property_name`` of the instance object.